In [1]:
import numpy as np
import pandas as pd
from steady_BEM import steady_BEM


In [2]:
# define flow conditions
Uinf = 10 # unperturbed wind speed in m/s
TSR = 10 # tip speed ratio
Radius = 50
N_blade_sec = 30
NBlades = 3

TipLocation_R =  1
RootLocation_R =  0.2
pitch = 2

airfoil = 'DU_polar.txt'

B = steady_BEM(airfoil, TipLocation_R, RootLocation_R, NBlades, Radius, Uinf, TSR, N_blade_sec)
CT,CP, results = B.get_solution(pitch)
print('CT: {}'.format(CT))
print('CP: {}'.format(CP))
    

CT: 0.763906108007987
CP: 0.4579578167787593
